In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hdramy-music-dataset/94_180.wav
/kaggle/input/hdramy-music-dataset/121_420.wav
/kaggle/input/hdramy-music-dataset/92_030.wav
/kaggle/input/hdramy-music-dataset/17_570.wav
/kaggle/input/hdramy-music-dataset/27_1080.wav
/kaggle/input/hdramy-music-dataset/29_390.wav
/kaggle/input/hdramy-music-dataset/62_510.wav
/kaggle/input/hdramy-music-dataset/28_270.wav
/kaggle/input/hdramy-music-dataset/125_000.wav
/kaggle/input/hdramy-music-dataset/25_060.wav
/kaggle/input/hdramy-music-dataset/44_270.wav
/kaggle/input/hdramy-music-dataset/114_270.wav
/kaggle/input/hdramy-music-dataset/50_390.wav
/kaggle/input/hdramy-music-dataset/113_150.wav
/kaggle/input/hdramy-music-dataset/111_000.wav
/kaggle/input/hdramy-music-dataset/101_270.wav
/kaggle/input/hdramy-music-dataset/89_090.wav
/kaggle/input/hdramy-music-dataset/116_330.wav
/kaggle/input/hdramy-music-dataset/53_150.wav
/kaggle/input/hdramy-music-dataset/121_090.wav
/kaggle/input/hdramy-music-dataset/98_180.wav
/kaggle/input/hdramy-musi

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd

In [ ]:
def is_good_sample(y, sr, min_duration=2.0, rms_threshold=0.01, zcr_threshold=(0.05, 0.3)): 
    # Check if the duration is above the minimum threshold
    duration = librosa.get_duration(y=y, sr=sr)
    if duration < min_duration:
        return False

    # Check if the RMS energy is above the threshold
    rms = np.mean(librosa.feature.rms(y=y))
    if rms < rms_threshold:
        return False

    # Check if the ZCR is within a reasonable range
    zcr = np.mean(librosa.feature.zero_crossing_rate(y))
    if not (zcr_threshold[0] <= zcr <= zcr_threshold[1]):
        return False

    return True